---
# Creating occurrence file

Read in the datafile we just created from `sound_production_to_presence.ipynb`

In [1]:
# df_presence.dtypes.values

```
array([dtype('<M8[ns]'), dtype('float64'), dtype('O'), dtype('O'),
       dtype('O'), dtype('O'), dtype('O'), dtype('<M8[ns]'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('O'), dtype('O'),
       dtype('O'), dtype('O'), dtype('int64'), dtype('<M8[ns]')],
      dtype=object)
```

In [1]:
import pandas as pd
import numpy as np

fname = 'data/sanctsound_presence.zip'

df_pres = pd.read_csv(fname,
                       #dtype = df_occur.dtypes.values, 
                       compression='zip')

df_pres['station'] = df_pres['dataset_id'].str.split("_",expand=True)[1]+"_"+df_pres['dataset_id'].str.split("_",expand=True)[2]

df_pres.sample(10)

C:\Users\Mathew.Biddle\AppData\Local\Temp\4\ipykernel_15376\3125043652.py:6: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pres = pd.read_csv(fname,


,start_time,dolphin_presence,dataset_id,WKT,decimalLatitude,decimalLongitude,vernacularName,time,bluewhale_presence,bluewhale_manual_presence,...,minkewhale_presence,plainfinmidshipman_presence,northatlanticrightwhale_presence,scientificName,scientificNameID,taxonRank,kingdom,propagationFrequency,eventDate,station
267562,NaN,NaN,noaaSanctSound_CI04_06_bluewhale,POINT (33.8489 -120.1174),33.84890,-120.117400,blue whale,2020-11-18 17:55:31.584000000,1.0,NaN,...,NaN,NaN,NaN,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,63,2020-11-18 17:55:31.584000000,CI04_06
592475,NaN,NaN,noaaSanctSound_MB03_04_bluewhale,POINT (36.37021 -122.314903),36.37021,-122.314903,blue whale,2020-12-08 12:58:25.120000000,1.0,NaN,...,NaN,NaN,NaN,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,63,2020-12-08 12:58:25.120000000,MB03_04
169681,NaN,NaN,noaaSanctSound_CI04_03_bluewhale,POINT (33.84888 -120.117),33.84888,-120.117000,blue whale,2019-10-05 17:14:25.840000000,1.0,NaN,...,NaN,NaN,NaN,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,63,2019-10-05 17:14:25.840000000,CI04_03
568408,NaN,NaN,noaaSanctSound_MB03_03_bluewhale,POINT (36.37021 -122.314903),36.37021,-122.314903,blue whale,2020-09-07 12:58:11.808000000,1.0,NaN,...,NaN,NaN,NaN,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,63,2020-09-07 12:58:11.808000000,MB03_03
426263,NaN,NaN,noaaSanctSound_MB01_06_bluewhale,POINT (36.7977 -121.9757),36.79770,-121.975700,blue whale,2020-10-17 06:52:02.880000000,1.0,NaN,...,NaN,NaN,NaN,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,63,2020-10-17 06:52:02.880000000,MB01_06
609157,2019-10-28 02:15:12.731000064,NaN,noaaSanctSound_CI03_02_bocaccio,POINT (33.48687 -119.01609),33.48687,-119.016090,bocaccio,NaN,NaN,NaN,...,NaN,NaN,NaN,Sebastes paucispinis,urn:lsid:marinespecies.org:taxname:274833,Species,Animalia,300,2019-10-28 02:15:12.731000064,CI03_02
585336,NaN,NaN,noaaSanctSound_MB03_04_bluewhale,POINT (36.37021 -122.314903),36.37021,-122.314903,blue whale,2020-11-07 01:00:41.952000256,1.0,NaN,...,NaN,NaN,NaN,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,63,2020-11-07 01:00:41.952000256,MB03_04
686605,2020-08-19 01:00:00,1.0,noaaSanctSound_SB02_11_dolphins_1h,POINT (42.47099 -70.24229),42.47099,-70.242290,dolphin,NaN,NaN,NaN,...,NaN,NaN,NaN,Cetacea,urn:lsid:marinespecies.org:taxname:2688,Infraorder,Animalia,5000,2020-08-19 01:00:00,SB02_11
543601,NaN,NaN,noaaSanctSound_MB03_02_bluewhale,POINT (36.37021 -122.314903),36.37021,-122.314903,blue whale,2019-10-04 10:19:37.736000000,1.0,NaN,...,NaN,NaN,NaN,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,63,2019-10-04 10:19:37.736000000,MB03_02
240892,NaN,NaN,noaaSanctSound_CI04_05_bluewhale,POINT (33.8489 -120.1171),33.84890,-120.117100,blue whale,2020-10-03 06:16:13.496000000,1.0,NaN,...,NaN,NaN,NaN,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,63,2020-10-03 06:16:13.496000000,CI04_05


# **To Do: Truncate down to the day**

For one day, was there an acoustic presence for x species.


In [2]:
cols = [col for col in df_pres.columns if 'presence' in col]

cols.extend([col for col in df_pres.columns if 'detection_count' in col])

cols
#df_occur_copy[cols].plot()

['dolphin_presence',
 'bluewhale_presence',
 'bluewhale_manual_presence',
 'bocaccio_presence',
 'finwhale_presence',
 'pinniped_presence',
 'seiwhale_presence',
 'atlanticcod_presence',
 'humpbackwhale_presence',
 'killerwhale_presence',
 'minkewhale_presence',
 'plainfinmidshipman_presence',
 'northatlanticrightwhale_presence',
 'redgrouper_detection_count',
 'blackgrouper_detection_count']

What we need to end up with is a table with the following information.

Species `x` made an acoustic sound at `y` location on `z` day.

Our sounding information is high resolution, so we want to group the sounds into daily sounds. But, we need to preserve the species, and location information.

Groupby DAY. This is doing the work of capturing daily observations (truncate by day).

Here we are grouping the dataframe by the species, spatial coordinates, and date.

### Groupby use .dt.date 

```
out2 = df_occur.groupby(df_occur.eventDate.dt.date)[cols].sum()

out2
```

In [ ]:
# df_pres['eventDate'] = pd.to_datetime(df_pres['eventDate'], format='%Y-%m-%d %H:%M:%S.%f')

# df_test = df_pres.set_index('eventDate')

# temp = df_test.loc['2021-09-25']

# temp.loc[(temp['vernacularName']=='dolphin') & (temp['station']=='SB02')]

In [3]:
df_pres['eventDate'] = pd.to_datetime(df_pres['eventDate'], format='%Y-%m-%d %H:%M:%S.%f')

df_pres.sort_values(by='eventDate', inplace=True)

group = df_pres.groupby(
    [df_pres.eventDate.dt.date,'station','vernacularName','scientificNameID','scientificName','taxonRank','kingdom','decimalLatitude','decimalLongitude','propagationFrequency']
)

counts = group[cols].count() # we just need to know if the species was present on that date and location.

counts = counts.reset_index(level=[0,1,2,3,4,5,6,7,8,9], allow_duplicates=True)

counts

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,propagationFrequency,...,pinniped_presence,seiwhale_presence,atlanticcod_presence,humpbackwhale_presence,killerwhale_presence,minkewhale_presence,plainfinmidshipman_presence,northatlanticrightwhale_presence,redgrouper_detection_count,blackgrouper_detection_count
0,2018-11-01,CI01_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0438,-120.0811,300,...,0,0,0,0,0,0,0,0,0,0
1,2018-11-01,CI01_01,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.0438,-120.0811,300,...,0,0,0,1,0,0,0,0,0,0
2,2018-11-01,CI01_01,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0438,-120.0811,300,...,0,0,0,0,0,0,1,0,0,0
3,2018-11-01,CI02_01,blue whale,urn:lsid:marinespecies.org:taxname:137090,Balaenoptera musculus,Species,Animalia,34.0856,-120.5232,63,...,0,0,0,0,0,0,0,0,0,0
4,2018-11-01,CI02_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0856,-120.5232,300,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22923,2021-12-31,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,...,0,0,0,0,0,0,0,0,0,0
22924,2022-01-02,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,...,0,0,0,0,0,0,0,0,0,0
22925,2022-01-03,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,...,0,0,0,0,0,0,0,0,0,0
22926,2022-01-04,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,...,0,0,0,0,0,0,0,0,0,0


In [4]:
counts['eventDate']=pd.to_datetime(counts['eventDate'])

counts = counts.set_index(counts['eventDate'])

counts

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,propagationFrequency,...,pinniped_presence,seiwhale_presence,atlanticcod_presence,humpbackwhale_presence,killerwhale_presence,minkewhale_presence,plainfinmidshipman_presence,northatlanticrightwhale_presence,redgrouper_detection_count,blackgrouper_detection_count
eventDate,,,,,,,,,,,,,,,,,,,,,
2018-11-01,2018-11-01,CI01_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0438,-120.0811,300,...,0,0,0,0,0,0,0,0,0,0
2018-11-01,2018-11-01,CI01_01,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.0438,-120.0811,300,...,0,0,0,1,0,0,0,0,0,0
2018-11-01,2018-11-01,CI01_01,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0438,-120.0811,300,...,0,0,0,0,0,0,1,0,0,0
2018-11-01,2018-11-01,CI02_01,blue whale,urn:lsid:marinespecies.org:taxname:137090,Balaenoptera musculus,Species,Animalia,34.0856,-120.5232,63,...,0,0,0,0,0,0,0,0,0,0
2018-11-01,2018-11-01,CI02_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0856,-120.5232,300,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,2021-12-31,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,...,0,0,0,0,0,0,0,0,0,0
2022-01-02,2022-01-02,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,...,0,0,0,0,0,0,0,0,0,0
2022-01-03,2022-01-03,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,...,0,0,0,0,0,0,0,0,0,0


Let's confirm that we have unique location, date, and species.

For one species, tell me if we have occurrences of other species. For this we use the describe() function to give us a summary of all the other presence variables. If the statistics are all 0 for the other columns, than we are doing this correctly.

In [5]:
counts.loc[counts['scientificNameID']=='urn:lsid:marinespecies.org:taxname:137090',cols].describe()

,dolphin_presence,bluewhale_presence,bluewhale_manual_presence,bocaccio_presence,finwhale_presence,pinniped_presence,seiwhale_presence,atlanticcod_presence,humpbackwhale_presence,killerwhale_presence,minkewhale_presence,plainfinmidshipman_presence,northatlanticrightwhale_presence,redgrouper_detection_count,blackgrouper_detection_count
count,2745.0,2745.000000,2745.000000,2745.0,2745.0,2745.0,2745.0,2745.0,2745.0,2745.0,2745.0,2745.0,2745.0,2745.0,2745.0
mean,0.0,217.943169,0.421129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,290.032984,6.299879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,17.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,92.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,314.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,1952.000000,194.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Cool! Looks like we are doing what we think we're doing. Now we can drop all of the presence columns, as we're identifying presence by indicating scientificNameID in the row. 

In [6]:
#columns = cols.extend(['eventDate'])

df_occur = counts.drop(columns=cols)

df_occur

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,propagationFrequency
eventDate,,,,,,,,,,
2018-11-01,2018-11-01,CI01_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0438,-120.0811,300
2018-11-01,2018-11-01,CI01_01,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.0438,-120.0811,300
2018-11-01,2018-11-01,CI01_01,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0438,-120.0811,300
2018-11-01,2018-11-01,CI02_01,blue whale,urn:lsid:marinespecies.org:taxname:137090,Balaenoptera musculus,Species,Animalia,34.0856,-120.5232,63
2018-11-01,2018-11-01,CI02_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0856,-120.5232,300
...,...,...,...,...,...,...,...,...,...,...
2021-12-31,2021-12-31,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20
2022-01-02,2022-01-02,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20
2022-01-03,2022-01-03,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20


Now we have an occurrence table! We have a species at a time and location.

Let's make an appropriate `occurrenceID` by appending the `vernacularName`, `station`, and `eventDate`.

Second we want to check that all occurrences are unique using this method. If the resultant table is empty, we have made locally unique identifiers.

In [7]:
df_occur['site'] = df_occur['station'].str.split("_",expand=True)[0]

df_occur['station'] = df_occur['station'].str.split("_",expand=True)[1]

df_occur['occurrenceID'] = 'site' + df_occur['site'] + "_" + 'station' + df_occur['station'] + '_' + df_occur['vernacularName'].str.replace(' ','_') + "_" + df_occur['eventDate'].dt.strftime('%Y-%m-%d')

df_occur.loc[df_occur['occurrenceID'].duplicated(keep=False)]

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,propagationFrequency,site,occurrenceID
eventDate,,,,,,,,,,,,


In [8]:
df_occur

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,propagationFrequency,site,occurrenceID
eventDate,,,,,,,,,,,,
2018-11-01,2018-11-01,01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0438,-120.0811,300,CI01,siteCI01_station01_bocaccio_2018-11-01
2018-11-01,2018-11-01,01,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.0438,-120.0811,300,CI01,siteCI01_station01_humpback_whale_2018-11-01
2018-11-01,2018-11-01,01,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0438,-120.0811,300,CI01,siteCI01_station01_plainfin_midshipman_2018-11-01
2018-11-01,2018-11-01,01,blue whale,urn:lsid:marinespecies.org:taxname:137090,Balaenoptera musculus,Species,Animalia,34.0856,-120.5232,63,CI02,siteCI02_station01_blue_whale_2018-11-01
2018-11-01,2018-11-01,01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0856,-120.5232,300,CI02,siteCI02_station01_bocaccio_2018-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,2021-12-31,07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,CI03,siteCI03_station07_fin_whale_2021-12-31
2022-01-02,2022-01-02,07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,CI03,siteCI03_station07_fin_whale_2022-01-02
2022-01-03,2022-01-03,07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,CI03,siteCI03_station07_fin_whale_2022-01-03


Let's write it out as an occurrence file.

In [9]:
#df_occur.dop(columns=['eventDate'], inplace=True) # we can drop eventDate column now (it's our index)

fname_occur = 'data/occurrence.zip'
df_occur.to_csv(fname_occur,date_format='%Y-%m-%d', index=False, compression='zip')